# Load Libraries

In [34]:
import xml.etree.ElementTree as ET
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
from sklearn import preprocessing

import zipfile
from glob import glob

import seaborn as sns
from sympy import subsets
sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

#import japanize_matplotlib
import unicodedata
import csv
import codecs
import glob


from MyMasterClass import MyMasterClass

In [35]:

class RECEPTClass(MyMasterClass):

    def __init__(self, data_dir=u"L:\\RECEPTY\\national",  output_dir = u"L:\\epson_pxm840f\\Shikakku"):
        super().__init__(data_dir, output_dir)


        filename = "RECEIPTY.CYO"
        self.nfilename = os.path.join(self.data_dir, filename)
        
        data_dir2=u"L:\\RECEPTY\\tokyo"
        self.tfilename = os.path.join(data_dir2, filename)
        

    def readData(self,filename):

        with codecs.open(filename, "r", "Shift-JIS", "ignore") as file:
            col_names = [ 'c{0:02d}'.format(i) for i in range(180) ]

            df_re = pd.read_csv(file, delimiter=",", names=col_names, engine="python" )
            #print(df_re.head())
            print("[ReceiptyClass] recepty csv data file size --> ",df_re.shape)

            return df_re
    
    def integrateInsurancedata(self):

        df_n = self.readData(self.nfilename)
        df_nat = self.makeInsurancedata(df_n)

        df_t = self.readData(self.tfilename)
        df_tok = self.makeInsurancedata(df_t)

        return pd.concat([df_nat,df_tok])
        
    def makeInsurancedata(self, df_RE):

        df_RE_index = df_RE[df_RE.c00 == "RE"].index.tolist() 

        retargetcols = ["c04","c05","c06","c07","c08"]
        hotargetcols = ["c01","c02","c03","c09"]
        kotargetcols = ["c01","c02","c03","c07","c09"]

        _re = df_RE[df_RE.c00 == "RE"][retargetcols].reset_index(drop=True)  
        _ho = df_RE[df_RE.c00 == "HO"][hotargetcols].reset_index(drop=True)
        _ko = df_RE[df_RE.c00 == "KO"][kotargetcols]# .reset_index(drop=True)

        df_list = []
        data1 = {"c01":np.nan,"c02":np.nan,"c03":np.nan,"c07":np.nan,"c09":np.nan }
        data2 = {"c01x":np.nan,"c02x":np.nan,"c03x":np.nan,"c07x":np.nan,"c09x":np.nan}

        # Kouhi loop (1)(2)
        ko_list1 = []
        ko_list2 = []

        for idx, re_idx in enumerate(df_RE_index[:-1]):
            
            ko_idx_list = []
            tmp_ko_idx = None
            for ko_idx in _ko.index.tolist():
                
                if ko_idx > re_idx and ko_idx <= df_RE_index[idx+1]:
                    try:
                        tmp_ko_idx = ko_idx
                        data1 = {"c01":_ko.c01,"c02":_ko.c02,"c03":_ko.c03,"c07":_ko.c03,"c09":_ko.c09 }
                        ko_idx_list.append(pd.DataFrame(data1, index=[ko_idx]) )
                    except:
                        print("Error : re idx %d ko_idx %d" % ( re_idx, ko_idx ))
                
            if len(ko_idx_list) == 1: # case 2nd kouhi not found dummy record added
                _tmp_df=pd.DataFrame(data2, index=[tmp_ko_idx])
                ko_idx_list.append( _tmp_df )
            
                _tmp = pd.concat( ko_idx_list,axis=1)
                #print(_tmp)

                ko_list1.append(_tmp)


        _df_ko_2 = pd.concat(ko_list1)    
        _df2 = pd.concat( [_re,_ho], axis=1)
        _df = pd.merge(_df2, _df_ko_2, how="left", left_index=True, right_index=True)

        _df.columns = ["Name","sex","birth","ratio","SpecialPurpose","InsurerNumber", 
                        "InsuredCardSymbol","InsuredIdentificationNumber", "InsuredFutan", 
                        "kouhi1","kouhi_jyu1","kyufu_kubu1","futan1","kouhifutan1",
                        "kouhi2","kouhi_jyu2","kyufu_kubu2","futan2","kouhifutan2" ]     

        _df['InsuredCardSymbol'] = [unicodedata.normalize("NFKC",str(z)) for z in _df['InsuredCardSymbol'].fillna("").apply(str)]
        _df['InsuredCardSymbol'] = [ s.replace('\u2010','-') for s in _df['InsuredCardSymbol'].tolist() ]
        _df['InsuredCardSymbol'] = [ s.replace('\u2212','-') for s in _df['InsuredCardSymbol'].tolist() ]
        _df['sex'] = [ s.replace('2',u'女') for s in _df['sex'].tolist() ]
        _df['sex'] = [ s.replace('1',u'男') for s in _df['sex'].tolist() ]
        _df['birth'] = _df['birth'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

        
        _df['InsuredIdentificationNumber'] = [unicodedata.normalize("NFKC",str(z)) for z in _df['InsuredIdentificationNumber'].fillna("").apply(str)]

        return _df.fillna("")


In [36]:
robj = RECEPTClass()
#df_RE = robj.readData()
df = robj.integrateInsurancedata()



[ReceiptyClass] recepty csv data file size -->  (5650, 180)
[ReceiptyClass] recepty csv data file size -->  (2373, 180)


In [37]:
ascii(df.loc[3,"InsuredCardSymbol"])

'3     89-0207\n3    38160031\nName: InsuredCardSymbol, dtype: object'

In [38]:
robj.toCsv(df,"test.csv")

## RECEPT data file 

### RE index

In [ ]:

df_RE[df_RE.c00 == "KO"]

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c170,c171,c172,c173,c174,c175,c176,c177,c178,c179
31,KO,88132196,4452009,NaN,1,341,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,KO,54136015,1011832,NaN,1,1019,NaN,2038.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,KO,82137530,2452530,NaN,2,2709,NaN,5418.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,KO,82138009,7871353,NaN,4,4135,NaN,NaN,NaN,8000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,KO,82138009,7002819,NaN,5,9315,NaN,NaN,NaN,8000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,KO,82138009,6349005,NaN,5,1264,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5262,KO,82138009,8237786,NaN,5,4810,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5381,KO,82138009,7037427,NaN,5,9922,NaN,NaN,NaN,10000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5457,KO,82138009,7847338,NaN,3,597,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
